In [ ]:
# 241026

from glob import glob
import os
import pandas as pd
import numpy as np
import json

path_list = r"path_list.csv"
answ_df = pd.read_csv(r'../0_data/answer_detailed.csv', index_col=0)

dtl_path = '.detailed'
out_path = f'{dtl_path}/resp'

In [2]:
def run_resp(resp_path, save_path, split_by):
    year_dict = {}
    with open(resp_path, 'rb') as f:
        exam = json.load(f)
    f.close()
    filename = os.path.basename(resp_path)
    
    resp_df = pd.DataFrame(index=range(1, 141))
    
    split_by = '@'
    alert_dict = {}
    symbol_to_number = {'①': 1, '②': 2, '③': 3, '④': 4, '⑤': 5}

    for i, response in enumerate(exam):
        year = exam[i]['year']
        subject = exam[i]['subject_id']
        ques_num = exam[i]['question_number']

        if exam[i]['subject_id'] == '1-1': ques_num -= 0
        elif exam[i]['subject_id'] == '1-2': ques_num -= 20
        elif exam[i]['subject_id'] == '1-3': ques_num -= 40
        elif exam[i]['subject_id'] == '1-4': ques_num -= 60
        elif exam[i]['subject_id'] == '1-5': ques_num -= 80
        elif exam[i]['subject_id'] == '2-1': ques_num -= 0
        elif exam[i]['subject_id'] == '2-2': ques_num -= 18
        elif exam[i]['subject_id'] == '2-3': ques_num -= 36
        elif exam[i]['subject_id'] == '2-4': ques_num -= 54
        elif exam[i]['subject_id'] == '2-5': ques_num -= 72
        elif exam[i]['subject_id'] == '3-1': ques_num -= 0
        elif exam[i]['subject_id'] == '3-2': ques_num -= 0
        elif exam[i]['subject_id'] == '3-3': ques_num -= 27
        elif exam[i]['subject_id'] == '3-4': ques_num -= 45
        elif exam[i]['subject_id'] == '4-1': ques_num -= 63
        response = exam[i]['response']
        col_name = f'{year}_{subject}'

        counts = {'①': 0, '②': 0, '③': 0, '④': 0, '⑤': 0}
        resp = None
        if split_by is not None:
            if split_by == '@':
                desc = str(response)
            else:
                desc = response.split(split_by)[1]
        for num in desc:
            if num in counts:
                counts[num] += 1
        
        first_occurrence = None
        for num in desc:
            if num in counts:
                first_occurrence = num
                break
        max_count = max(counts.values())
        max_elements = [key for key, value in counts.items() if value == max_count]
        if len(max_elements) == 1:
            mode = max_elements[0]
        else:
            mode = None

        if first_occurrence == mode:
            resp = first_occurrence
        else:
            if mode is None:
                resp = first_occurrence
            else:
                resp = mode
            alert_dict[i+1] = desc

        if resp is not None:
            resp_numb = symbol_to_number[resp]
        else:
            for num in desc:
                if num in '12345':
                    resp = num
                    break
            if resp is not None:
                resp_numb = int(resp)
            else:
                resp_numb = None
                alert_dict[i+1] = desc
        
        if col_name not in resp_df.columns:
            resp_df[col_name] = None
        if resp_numb is not None:
            resp_df.at[ques_num, col_name] = resp_numb
        
    year_dict[filename] = alert_dict
    resp_df = resp_df.drop(index=range(78, 141))
    resp_df.to_csv(f'.detailed/resp/{save_path}_response.csv')

    return resp_df, year_dict

In [ ]:
data_df = pd.read_csv(path_list)
mean_df = pd.DataFrame()
# dtl_path = '.detailed'
# out_path = f'{dtl_path}/resp'
if not os.path.exists(out_path):
    os.makedirs(out_path)
conv_dict = {}
for i in range(len(data_df)):
    resp_path = data_df['path'][i]
    save_path = data_df['name'][i]
    split_by = data_df['split'][i]
    
    resp_df, year_dict = run_resp(resp_path, save_path, split_by)

In [4]:
def make_score(lang):
    resp_patrn = rf".detailed/resp/*{lang}]_response.csv"
    resp_paths = glob(resp_patrn)
    print(len(resp_paths))

    score_path = f'{dtl_path}/score'
    os.makedirs(score_path, exist_ok=True)

    for i, path in enumerate(resp_paths):
        resp_df = pd.read_csv(path, index_col=0)
        name = os.path.basename(path)
        name = name.split('_response')[0]
        common_nan_mask = answ_df.isna() & resp_df.isna()

        score_df = pd.DataFrame(np.where(answ_df.eq(resp_df), 1, 0), index=answ_df.index, columns=answ_df.columns)
        score_df[common_nan_mask] = np.nan
        score_df.to_csv(f'{score_path}/{name}.csv')

lang = ['eng', 'kor']
for l in lang:
    make_score(l)

38
38
